In [346]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import re
import pymongo

In [347]:
def finding_raw_urls_from_base(url):
    service = Service("./chromedriver/chromedriver.exe")
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    
    driver = webdriver.Chrome(service=service, options=options)
    driver.get(url)
    
    WebDriverWait(driver, 60).until(
        EC.presence_of_all_elements_located((By.TAG_NAME, 'a'))
    )
    
    html = driver.page_source
    driver.quit()
    
    soup = BeautifulSoup(html, 'html.parser')
    links = soup.find_all('a', href=True)
    raw_urls = []
    for link in links:
        if link['href'].startswith("https://www.moneycontrol.com/news/business/"):
            raw_urls.append(link['href'])
    return list(set(raw_urls))
    

In [348]:
# raw_urls = finding_raw_urls_from_base("https://www.moneycontrol.com/news/business/stocks/")
raw_urls = finding_raw_urls_from_base("https://www.moneycontrol.com/news/business/stocks/page-12")

In [349]:
raw_urls

['https://www.moneycontrol.com/news/business/markets/taking-stock-friday-blockbuster-sensex-nifty-recover-more-than-2-from-day-s-low-12889373.html',
 'https://www.moneycontrol.com/news/business/stocks/page-8/',
 'https://www.moneycontrol.com/news/business/markets/stock-market-live-sensex-nifty-50-share-price-gift-nifty-latest-updates-27-12-2024-liveblog-12898541.html',
 'https://www.moneycontrol.com/news/business/markets/indian-oil-shares-rise-3-as-jefferies-upgrades-to-buy-sees-31-upside-12889085.html',
 'https://www.moneycontrol.com/news/business/markets/bharti-airtel-stock-surges-4-biggest-jump-in-7-weeks-12889278.html',
 'https://www.moneycontrol.com/news/business/stocks/page-10/',
 'https://www.moneycontrol.com/news/business/',
 'https://www.moneycontrol.com/news/business/stocks/page-15/',
 'https://www.moneycontrol.com/news/business/companies/',
 'https://www.moneycontrol.com/news/business/markets/india-s-stock-market-growth-is-a-testament-to-investor-trust-in-indian-businesses-a

In [350]:
stock_urls =[]
market_urls = []
next_page_urls = []
other_urls = []
ipo_urls = []
def extract_urls(all_urls):
    for link in raw_urls:
        if link.startswith("https://www.moneycontrol.com/news/business/stocks/page"):
            next_page_urls.append(link)
        elif link.startswith("https://www.moneycontrol.com/news/business/stocks/"):
            stock_urls.append(link)
        elif link.startswith("https://www.moneycontrol.com/news/business/markets/"):
            market_urls.append(link)
        elif link.startswith("https://www.moneycontrol.com/news/business/ipo/"):
            ipo_urls.append(link)
        else:
            other_urls.append(link)

In [351]:
extract_urls(raw_urls)

In [352]:
stock_urls

['https://www.moneycontrol.com/news/business/stocks/subscribe-for-international-gemmological-institute-india-ltd-ipo-anand-rathi-12889293.html',
 'https://www.moneycontrol.com/news/business/stocks/broader-indices-underperform-but-these-smallcaps-gain-between-10-37-12889827.html',
 'https://www.moneycontrol.com/news/business/stocks/buy-polycab-india-target-of-rs-8741-prabhudas-lilladher-12889399.html',
 'https://www.moneycontrol.com/news/business/stocks/weekly-tactical-pick-why-this-niche-fmcg-stock-merits-attention-post-correction-12889091.html',
 'https://www.moneycontrol.com/news/business/stocks/buy-bajaj-finserv-target-of-rs-2350-sharekhan-2-12889213.html',
 'https://www.moneycontrol.com/news/business/stocks/buy-persistent-systems-target-of-rs-7280-sharekhan-12889262.html',
 'https://www.moneycontrol.com/news/business/stocks/']

In [353]:

regex = r'https:\/\/www\.moneycontrol\.com\/news\/business\/stocks\/[^\/\s]+(?:\.[a-z]{2,6})(?:[\/\?].*)?'

final_stocks_urls = [url for url in stock_urls if re.match(regex, url)]

In [354]:
final_stocks_urls

['https://www.moneycontrol.com/news/business/stocks/subscribe-for-international-gemmological-institute-india-ltd-ipo-anand-rathi-12889293.html',
 'https://www.moneycontrol.com/news/business/stocks/broader-indices-underperform-but-these-smallcaps-gain-between-10-37-12889827.html',
 'https://www.moneycontrol.com/news/business/stocks/buy-polycab-india-target-of-rs-8741-prabhudas-lilladher-12889399.html',
 'https://www.moneycontrol.com/news/business/stocks/weekly-tactical-pick-why-this-niche-fmcg-stock-merits-attention-post-correction-12889091.html',
 'https://www.moneycontrol.com/news/business/stocks/buy-bajaj-finserv-target-of-rs-2350-sharekhan-2-12889213.html',
 'https://www.moneycontrol.com/news/business/stocks/buy-persistent-systems-target-of-rs-7280-sharekhan-12889262.html']

In [355]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["new_db"]
mycol = mydb['raw_news']

In [356]:
service = Service("./chromedriver/chromedriver.exe")
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.199 Safari/537.36")

driver = webdriver.Chrome(service=service, options= options)

In [357]:
def extract_data(url):  
    driver.get(url)
    
    WebDriverWait(driver, 60).until(
        lambda d: d.execute_script("return document.readyState") == "complete"
    )
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')    
    news = {}
    
    title = soup.find('h1', class_ = "article_title")
    news.update({"title":f"{title.text.strip()}"})
    
    desc = soup.find('h2', class_ = 'article_desc')
    news.update({"desc":f"{desc.text.strip()}"})
    
    date_time_div = soup.find('div', class_="article_schedule")
    if date_time_div:
        span_tag = date_time_div.find('span')
        date = span_tag.text.strip()
        news.update({"date":f"{date}"})
    
    datetime = date_time_div.text.strip()
    news.update({"datetime":f"{datetime}"})
    
    paragraphs_list = []
    paragrphs_div = soup.find('div', class_ ="content_wrapper")
    if paragrphs_div:
        paragraph_tags = paragrphs_div.find_all('p')
        for p in paragraph_tags:
            para_text = p.text.strip()            
            if len(para_text) < 50 :
                continue            
            if re.search(r"(click\s+here|disclaimer|modal|window|advertisement|investment\s+tips)", para_text, re.IGNORECASE):
                continue            
            paragraphs_list.append(para_text)         
            
    news.update({"content": paragraphs_list})
            
    # driver.quit()   
    return news

In [358]:
for url in final_stocks_urls :
    data = extract_data(url)
    x = mycol.insert_one(data)
    print(x.inserted_id)

676ea06249f67e6ab04c1f2e
676ea06a49f67e6ab04c1f2f
676ea06f49f67e6ab04c1f30
676ea07349f67e6ab04c1f31


In [345]:
driver.quit()   